In [13]:
# !pip install keras_vggface
# !pip install keras_applications
# !pip install deepface

In [14]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, LayerNormalization, BatchNormalization, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2

from deepface.commons.functions import find_input_shape, normalize_input
from deepface.DeepFace import build_model

from tqdm import tqdm

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [30]:
# Hyperparameters
BASE_MODEL = 'OpenFace'
IGNORE_TOP_NLAYERS_ARCH = 5
IGNORE_BOTTOM_NLAYERS_TUNE = 1
IGNORE_TOP_NLAYERS_TUNE = 1
NORMALIZATION = 'base'
FINE_TUNE = False

# Modify paths as per your method of saving them
BASE_PATH = "/root/KinshipRecognition"
TRAIN_FILE_PATH = f"{BASE_PATH}/data/aug_train_ds.csv"
TRAIN_FOLDERS_PATH = f"{BASE_PATH}/data/train/train-faces/"

# All images belonging to families F09** will be used to create the validation set while training the model
# For final submission, you can add these to the training data as well
val_families_list = ["F02"]
# val_families_list = ["F02","F04","F06","F08", "F09"]

# Output file
MODEL_NAME = f"ensemble_deepface_{BASE_MODEL}_notune_dense32-128-32_drop05"

# Get input shape and normalization method.
INPUT_SHAPE = find_input_shape(build_model(BASE_MODEL))
NORMALIZATION = 'base' 

In [31]:
def get_train_val(family_name):

    val_families = family_name

    all_images = glob(TRAIN_FOLDERS_PATH + "*/*/*.jpg")
    train_images = [x for x in all_images if val_families not in x]
    val_images = [x for x in all_images if val_families in x]

    train_person_to_images_map = defaultdict(list)

    ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

    for x in train_images:
        train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

    val_person_to_images_map = defaultdict(list)

    for x in val_images:
        val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    relationships = pd.read_csv(TRAIN_FILE_PATH)
    relationships = list(zip(relationships.p1.values, relationships.p2.values, relationships.relationship.values))
    relationships = [(x[0],x[1],x[2]) for x in relationships if x[0][:10] in ppl and x[1][:10] in ppl]    

    train = [x for x in relationships if val_families not in x[0]]
    val = [x for x in relationships if val_families in x[0]]
    return train, val, train_person_to_images_map, val_person_to_images_map

In [32]:
def read_img(path, input_shape, normalization='base'):
    img = cv2.imread(path, -1)
    img = cv2.resize(img, input_shape)
    img = cv2.normalize(img,  np.zeros(img.shape[:2]), 0, 255, cv2.NORM_MINMAX)
    img = normalize_input(img, normalization=NORMALIZATION)
    return np.array(img).astype(np.float)

In [33]:
def gen(list_tuples, person_to_images_map, input_shape, batch_size=16, normalization='base'):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size)
        
        # All the samples are taken from train_ds.csv, labels are in the labels column
        labels = []
        for tup in batch_tuples:
            labels.append(tup[2])
        labels = np.array(labels)

        # Original images preprocessed
        X1 = [x[0] for x in batch_tuples]
        X1 = np.array([read_img(TRAIN_FOLDERS_PATH + x, input_shape) for x in X1])
        
        X2 = [x[1] for x in batch_tuples]
        X2 = np.array([read_img(TRAIN_FOLDERS_PATH + x, input_shape) for x in X2])
        
        # Mirrored images
        X1_mirror = np.asarray([cv2.flip(x, 1) for x in X1])
        X2_mirror = np.asarray([cv2.flip(x, 1) for x in X2])
        X1 = np.r_[X1, X1_mirror]
        X2 = np.r_[X2, X2_mirror]
        
        yield [X1, X2], np.r_[labels, labels]

In [34]:
def baseline_model(model_name, fine_tune=True):
    input_1 = Input(shape=INPUT_SHAPE + (3,))
    input_2 = Input(shape=INPUT_SHAPE + (3,))
    
    backbone = build_model(BASE_MODEL)
    backbone = Model(backbone.layers[0].input, backbone.layers[-IGNORE_TOP_NLAYERS_ARCH].output)
    for x in backbone.layers:
        x.trainable = False

    if fine_tune:
        for x in backbone.layers[:IGNORE_BOTTOM_NLAYERS_TUNE]:
            x.trainable = False
        if IGNORE_TOP_NLAYERS_TUNE == 0:
            for x in backbone.layers[IGNORE_BOTTOM_NLAYERS_TUNE:]:
                x.trainable = True
        else:
            for x in backbone.layers[IGNORE_BOTTOM_NLAYERS_TUNE:-IGNORE_TOP_NLAYERS_TUNE]:
                x.trainable = True

    for x in backbone.layers:
        print(x.name, x.trainable)
                
    x1 = backbone(input_1)
    x2 = backbone(input_2)

    x1 = Flatten()(x1)
    x2 = Flatten()(x2)

    x1 = LayerNormalization(axis=-1, epsilon=0.001, center=False, scale=False)(x1)
    x2 = LayerNormalization(axis=-1, epsilon=0.001, center=False, scale=False)(x2)

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])
    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4 = Subtract()([x1_, x2_])
    x5 = Multiply()([x1, x2])
    x = Concatenate(axis=-1)([x3, x4, x5])
        
#     x = LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.05)(x)    
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.05)(x)    
    x = Dense(32, activation="tanh")(x)
#     x = LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=False)(x)
    x = Dropout(0.05)(x)    
    out = Dense(1, kernel_regularizer=L2(.01), activation="sigmoid")(x)

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00002))
    model.summary()

    return model


In [35]:
# Print layers of backbone
backbone = build_model(BASE_MODEL)
backbone = Model(backbone.layers[0].input, backbone.layers[-IGNORE_TOP_NLAYERS_ARCH].output)
for x in backbone.layers:
    print(x.name)

input_4
zero_padding2d
conv1
bn1
activation
zero_padding2d_1
max_pooling2d
lrn_1
conv2
bn2
activation_1
zero_padding2d_2
conv3
bn3
activation_2
lrn_2
zero_padding2d_3
max_pooling2d_1
inception_3a_3x3_conv1
inception_3a_5x5_conv1
inception_3a_3x3_bn1
inception_3a_5x5_bn1
activation_3
activation_5
max_pooling2d_2
zero_padding2d_4
zero_padding2d_5
inception_3a_pool_conv
inception_3a_3x3_conv2
inception_3a_5x5_conv2
inception_3a_pool_bn
inception_3a_1x1_conv
inception_3a_3x3_bn2
inception_3a_5x5_bn2
activation_7
inception_3a_1x1_bn
activation_4
activation_6
zero_padding2d_6
activation_8
concatenate_1
power2_3b
inception_3b_3x3_conv1
inception_3b_5x5_conv1
average_pooling2d
inception_3b_3x3_bn1
inception_3b_5x5_bn1
mult9_3b
activation_9
activation_11
sqrt_3b
zero_padding2d_7
zero_padding2d_8
inception_3b_pool_conv
inception_3b_3x3_conv2
inception_3b_5x5_conv2
inception_3b_pool_bn
inception_3b_1x1_conv
inception_3b_3x3_bn2
inception_3b_5x5_bn2
activation_13
inception_3b_1x1_bn
activation_10


In [36]:
for i in range(len(val_families_list)):

    print('##############################')
    print(f'Iteration {i}: Validation on {val_families_list[i]}')
    print('##############################')

    train, val, train_person_to_images_map, val_person_to_images_map = get_train_val(val_families_list[i])
    file_path = f"{BASE_PATH}/log/model/{MODEL_NAME}_{i}.h5"
    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.3, patience=30, verbose=1)
    callbacks_list = [checkpoint, reduce_on_plateau]
    
    model = baseline_model(BASE_MODEL, fine_tune=FINE_TUNE)
    
    history = model.fit(gen(train, train_person_to_images_map, INPUT_SHAPE, batch_size=16), 
                        validation_data=gen(val, val_person_to_images_map, INPUT_SHAPE, batch_size=16), 
                        epochs=25, steps_per_epoch=300, validation_steps=200,
                        verbose=1, callbacks=callbacks_list, 
                        use_multiprocessing=False, workers=1)

##############################
Iteration 0: Validation on F02
##############################
input_4 False
zero_padding2d False
conv1 False
bn1 False
activation False
zero_padding2d_1 False
max_pooling2d False
lrn_1 False
conv2 False
bn2 False
activation_1 False
zero_padding2d_2 False
conv3 False
bn3 False
activation_2 False
lrn_2 False
zero_padding2d_3 False
max_pooling2d_1 False
inception_3a_3x3_conv1 False
inception_3a_5x5_conv1 False
inception_3a_3x3_bn1 False
inception_3a_5x5_bn1 False
activation_3 False
activation_5 False
max_pooling2d_2 False
zero_padding2d_4 False
zero_padding2d_5 False
inception_3a_pool_conv False
inception_3a_3x3_conv2 False
inception_3a_5x5_conv2 False
inception_3a_pool_bn False
inception_3a_1x1_conv False
inception_3a_3x3_bn2 False
inception_3a_5x5_bn2 False
activation_7 False
inception_3a_1x1_bn False
activation_4 False
activation_6 False
zero_padding2d_6 False
activation_8 False
concatenate_1 False
power2_3b False
inception_3b_3x3_conv1 False
inception_3b

300/300 [==============================] - 20s 52ms/step - loss: 0.7207 - acc: 0.6052 - val_loss: 0.6451 - val_acc: 0.6939

Epoch 00001: val_acc improved from -inf to 0.69391, saving model to /root/KinshipRecognition/log/model/ensemble_deepface_OpenFace_notune_dense32-128-32_drop05_0.h5
Epoch 2/25
300/300 [==============================] - 14s 46ms/step - loss: 0.6633 - acc: 0.6606 - val_loss: 0.6332 - val_acc: 0.6981

Epoch 00002: val_acc improved from 0.69391 to 0.69813, saving model to /root/KinshipRecognition/log/model/ensemble_deepface_OpenFace_notune_dense32-128-32_drop05_0.h5
Epoch 3/25
300/300 [==============================] - 14s 47ms/step - loss: 0.6415 - acc: 0.6833 - val_loss: 0.6399 - val_acc: 0.6853

Epoch 00003: val_acc did not improve from 0.69813
Epoch 4/25
300/300 [==============================] - 14s 48ms/step - loss: 0.6540 - acc: 0.6602 - val_loss: 0.6462 - val_acc: 0.6741

Epoch 00004: val_acc did not improve from 0.69813
Epoch 5/25
300/300 [====================

In [37]:
test_path = f"{BASE_PATH}/data/test/"
submission = pd.read_csv(f'{BASE_PATH}/data/test_ds.csv')
preds_for_sub = np.zeros(submission.shape[0])
all_preds = list()
for i in range(len(val_families_list)):

    print('##############################')
    print(f'Iteration {i}: Validation on {val_families_list[i]}')
    print('##############################')
    
#     model = baseline_model(BASE_MODEL, fine_tune=FINE_TUNE)
#     file_path = f"{BASE_PATH}/log/model/{MODEL_NAME}_{i}.h5"
#     model.load_weights(file_path)

    # Predictions
    predictions = []
    for j in range(0, len(submission.p1.values), 32):
        X1 = submission.p1.values[j:j+32]
        X1 = np.array([read_img(test_path + x, INPUT_SHAPE) for x in X1])

        X2 = submission.p2.values[j:j+32]
        X2 = np.array([read_img(test_path + x, INPUT_SHAPE) for x in X2])

        pred = model.predict([X1, X2]).ravel().tolist()
        predictions += pred    
    
    all_preds.append(np.array(predictions))
    preds_for_sub += np.array(predictions) / len(val_families_list)

    
all_preds = np.asarray(all_preds)
submission['score'] = preds_for_sub
pd.DataFrame(all_preds).to_csv(f"{BASE_PATH}/log/results/{MODEL_NAME}_allpreds.csv", index=False)
submission.to_csv(f"{BASE_PATH}/log/results/{MODEL_NAME}.csv", index=False)

##############################
Iteration 0: Validation on F02
##############################


In [40]:
print(np.sum(preds_for_sub <= 0.5))
print(len(preds_for_sub), '\n')
for line in preds_for_sub:
    print(line)

2478
3000 

0.06074224412441254
0.5791699886322021
0.30794626474380493
0.4724733829498291
0.2630940079689026
0.18399707973003387
0.5343971252441406
0.09604781866073608
0.18403130769729614
0.5038120746612549
0.2579983174800873
0.42678961157798767
0.25028255581855774
0.14362947642803192
0.2799299359321594
0.5851382613182068
0.39961957931518555
0.35752245783805847
0.33764898777008057
0.33556193113327026
0.10514041781425476
0.42267706990242004
0.308778315782547
0.4162258803844452
0.5674288868904114
0.24088981747627258
0.2936837375164032
0.2814095616340637
0.14209619164466858
0.11853363364934921
0.5757107138633728
0.4930933713912964
0.2784790098667145
0.3287982940673828
0.24285364151000977
0.19007067382335663
0.35307449102401733
0.2653328776359558
0.32410386204719543
0.2927558422088623
0.3904809355735779
0.28528398275375366
0.6234858632087708
0.09976118803024292
0.5154961347579956
0.25591713190078735
0.38707032799720764
0.3906501233577728
0.33496570587158203
0.11155958473682404
0.1960933357